<a href="https://colab.research.google.com/github/verticalmeadows/FreeCodeCamp-Pandas-Real-Life-Example/blob/master/up_to_geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Syntax in Python

In [ ]:
# set the midpoint
midpoint = 5

# make two empty lists
lower = []; upper = []

# split the numbers into lower and upper
for i in range(10):
    if (i < midpoint):
        lower.append(i)
    else:
        upper.append(i)

print("lower:", lower)
print("upper:", upper)

lower: [0, 1, 2, 3, 4]
upper: [5, 6, 7, 8, 9]


In [ ]:
x = 3
y = 4
if x < 4 :
  print(str(x)+ ' is small.')
print(y)

3 is small.
4


In [ ]:
x = 1 + 2 + 3 \ # backslash works in markdown as well
+ 4 + 5
print(x)
x

15


15

In [ ]:
print(x,y,"they are arguments")

15 4 they are arguments


# Semantics

no need to explicitly specify the type of our variables. It is assigned automatically. As it is a dynamically typed language.


In [ ]:
type(x)

int

In [ ]:
z = 3.4
type(z)

float

In [ ]:
x.conjugate

<function int.conjugate>

when typing "." after a variable, leike x. we can see the different functions and stuff that are related to the specific type.\
dir(x) shows us what can be done with a specific thing
same with dir(int)
functions are purple squares in the instant popup list

we can get help by typing ? after some function
e.g. x.conjugate?

In [ ]:
x.conjugate?

In [ ]:
x = "csakany"
x = 1
x = [1, 2, 3]

In [ ]:
x = [1, 2, 3]
y = x
print(x)
print(y)

[1, 2, 3]
[1, 2, 3]


In [ ]:
type(x) # but x.type() wouldn't work

list

In [ ]:
x.append?

In [ ]:
x.append(4)
print(y)
print(x)

[1, 2, 3, 4]
[1, 2, 3, 4]


there are mutable and immutable objects. Mutable ones are what?
Immutable ones when created are completely new.

Check how they behave. .append doens't change the pointer, just what's sitting at the memory place. x = x + 1 completely reassigns

In [ ]:
x = 3
y = x
print(x)
print(y)

3
3


In [ ]:
x = x + 1 # thisi is a reassignment, I'm changing the pointer in the memory. Y still points to the same memory lcation as before. Because integers are immutable values. So basically we create a new object
print(x)
print(y)

5
3


# Title

python script to write out polygon complexity  based on https://github.com/sical/polygons_complexity/blob/master/complexity.py

In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 8.2 MB/s 
     |████████████████████████████████| 6.3 MB 42.5 MB/s 
     |████████████████████████████████| 16.7 MB 21.8 MB/s 


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 31 15:28:44 2018
@author: thomas
"""

import math
import os
import pandas as pd
import geopandas as gpd
from tabulate import tabulate as tb


def get_notches(poly):
    """
    Determine the number of notches in a polygon object and calculate
    normalized notches of polygon

    Based on:
        "Measuring the Complexity of Polygonal Objects"
        (Thomas Brinkhoff, Hans-Peter Kriegel, Ralf Schneider, Alexander Braun)
        http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.73.1045&rep=rep1&type=pdf

        https://github.com/pondrejk/PolygonComplexity/blob/master/PolygonComplexity.py

    @poly (Shapely Polygon object)

    Returns normalized notches
    """
    notches = 0
    coords = list(poly.exterior.coords)
    for i, pt in enumerate(coords[:-1]):
        x_diff = coords[i+1][0] - pt[0]
        y_diff = coords[i+1][1] - pt[1]
        angle = math.atan2(y_diff, x_diff)
        if angle > math.pi:
            notches += 1

    if notches != 0:
        notches_norm = notches / (len(coords)-3)
    else:
        notches_norm = 0

    return notches_norm

def get_stats(gdf, coeff_ampl, coeff_conv):
    """
    Get polygon's amplitude of vibration:

    ampl(pol) = (boundary(pol) - boundary(convexhull(pol))) / boundary(pol)

    Get deviation from convex hull:
    conv(pol) = (area(convexhull(pol)) - area(pol)) / area(convexhull(pol))

    Measure complexity

     Based on:
        "Measuring the Complexity of Polygonal Objects"
        (Thomas Brinkhoff, Hans-Peter Kriegel, Ralf Schneider, Alexander Braun)
        http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.73.1045&rep=rep1&type=pdf

        https://github.com/pondrejk/PolygonComplexity/blob/master/PolygonComplexity.py

    Get area, centroid, distance from each others, boudary, convex hull,
    perimeter, number of vertices

    @param gdf(GeoDataframe): geodataframe with polygons
    @param coeff_ampl(float): coefficient for amplitude's calculation
    @param coeff_conv(float): coefficient for deviation from convex hull calculation

    Returns tuple with dict of stats values and GeoDataframe with stats

    """
    nb = gdf['geometry'].count()
    gdf['area'] = gdf['geometry'].area
    tot_area = gdf['area'].sum()
    gdf['centroid'] = gdf['geometry'].centroid
#    gdf['distance'] = gdf['geometry'].distance()
    gdf['boundary'] = gdf['geometry'].boundary
    gdf['convex_hull'] = gdf['geometry'].convex_hull
    gdf['convex_boundary'] = gdf['geometry'].convex_hull.boundary
    gdf['convex_area'] = gdf['geometry'].convex_hull.area
    gdf['nbvertices'] = gdf['geometry'].apply(lambda x: len(list(x.exterior.coords)))
    gdf['notches'] = gdf['geometry'].apply(lambda x: get_notches(x))

    gdf['amplitude'] = gdf.apply(
            lambda x:(
                    x['boundary'].length - x['convex_boundary'].length
                    ) / x['boundary'].length,
                    axis=1)
    gdf['convex'] = gdf.apply(
            lambda x: (
                    x['convex_area'] - x['area']
                    ) / x['convex_area'],
                    axis=1)
    gdf['complexity'] = gdf.apply(
            lambda x: coeff_ampl*x['amplitude'] * x['notches'] + coeff_conv * x['convex'],
            axis=1
            )

    mean_amplitude = gdf['amplitude'].mean()
    mean_convex = gdf['convex'].mean()
    mean_norm_notches = gdf['notches'].mean()
    mean_complexity = gdf['complexity'].mean()

    gdf['perimeter'] = gdf['geometry'].length
    tot_perimeter = gdf['perimeter'].sum()

    if ("lat" in gdf.columns) or ("lon" in gdf.columns):
        columns_drop = ["boundary", "convex_hull", "convex_boundary", "convex_area", "centroid", "lat", "lon"]
    else:
        columns_drop = ["boundary", "convex_hull", "convex_boundary", "convex_area", "centroid"]
    gdf = gdf.drop(columns_drop, axis=1)

    gdf = gdf.reset_index()

    if nb > 1:
        gdf = gdf.sort_values(by='perimeter', ascending=False)
        gdf = gdf.iloc[[0]]

    return {
            'area':tot_area,
            'perimeter':tot_perimeter,
#            'distance':mean_distance,
            'nb':nb,
            'amplitude': mean_amplitude,
            'convex': mean_convex,
            'notches': mean_norm_notches,
            'complexity': mean_complexity
            }, gdf

def complexity(shapes, images, coeff_ampl, coeff_conv, str_img):
    """
    @param shapes(): glob directory with shapefiles
    @param images(): glob directory with image files
    @param coeff_ampl(float): coefficient for amplitude's calculation
    @param coeff_conv(float): coefficient for deviation from convex hull calculation
    @param str_img(str): string of image filepath

    Returns all Polygons with stats in a GeoDatafame
    """
    l_gdf = []
    pd.options.display.float_format = '{:,.2f}'.format
    for shape in shapes:
        gdf = gpd.GeoDataFrame.from_file(shape)
        dict_complexity, gdf = get_stats(gdf, coeff_ampl, coeff_conv)
        name = os.path.basename(shape)
        name = name.replace(".shp","")
        gdf['name'] = name
        gdf['img'] = ''
#        gdf = img_str(gdf, images, name, str_img)

        gdf = gdf.drop('geometry', axis=1)
        l_gdf.append(gdf)

    gdf_tot = pd.concat(l_gdf)

    return gdf_tot

def img_str(x, str_img):
    x["img"] = str_img.format(x["name"])

    return x

def to_table(df, tablefmt, filename, str_img):
    """
    Export dataframe to a table file with a specific format
    (see tabulate doc for more information:
        https://pypi.org/project/tabulate/):
        - "plain"
        - "simple"
        - "grid"
        - "fancy_grid"
        - "pipe"
        - "orgtbl"
        - "jira"
        - "presto"
        - "psql"
        - "rst"
        - "mediawiki"
        - "moinmoin"
        - "youtrack"
        - "html"
        - "latex"
        - "latex_raw"
        - "latex_booktabs"
        - "textile"

    @param df(dataframe): dataframe with stats and img
    @tablefmt(str): format of the table
    @filename(str): path and name to file
    """
    df = df.apply(lambda x: img_str(x, str_img), axis=1)

    f = open(filename, 'w')
    f.write(tb(
            df,
            headers="keys",
            showindex=False,
            tablefmt=tablefmt
            ))
    f.close()

And the actual code that uses some of these functions defined in the previous chunk

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  5 10:50:24 2018
@author: thomas
"""
import glob
from complexity import complexity, to_table

if __name__ == "__main__":
    dir_shapes = ".\\shapes\\*.shp"
    dir_img = ".\\images\\*.png"
    shapes = glob.glob(dir_shapes)
    images = glob.glob(dir_img)
    coeff_ampl, coeff_conv = 0.8, 0.2

    #Export to latex
    filename = "complexity.tex"
    tablefmt = "latex_raw"
    str_img = "\includegraphics[width=1cm]{{figures/{}.png}}"

    gdf = complexity(shapes, images, coeff_ampl, coeff_conv, str_img)
    to_table(gdf, tablefmt, filename, str_img)

    #Export to html
    filename = "complexity.html"
    tablefmt = "html"
    str_img = '<img src=".//images//{}.png" style="width:10%;">'

    to_table(gdf, tablefmt, filename, str_img)